In [1]:
from franc_lib.semantics import Semantics
from franc_lib.lexical import Preprocessing

/home/franciscone/.envs/nlp/lib/python3.5/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [2]:
s = Semantics()

In [3]:
s.sentiment_analysis(text="Essa geladeira é ruim")

Trainning....
[LibLinear]

/home/franciscone/.envs/nlp/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/franciscone/.envs/nlp/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/franciscone/.envs/nlp/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:1544: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 3.
  " = {}.".format(effective_n_jobs(self.n_jobs)))


Finished !
Precisão 55.5204
[[  20   16    3   12   60   38]
 [   1  105   46   66  125  123]
 [   0   47   58  128  240   79]
 [   4   27   25  281 1191  271]
 [   0    9    5  151 3219 1656]
 [   2    5    4   25 1547 3689]]


array([2.])

In [4]:
s.get_confusion_matrix()
# s.classifier.predict(s.X_test)

v/v	 0.0	 1.0	 2.0	 3.0	 4.0	 5.0
0.0	 20	 16	 3	 12	 60	 38	
1.0	 1	 105	 46	 66	 125	 123	
2.0	 0	 47	 58	 128	 240	 79	
3.0	 4	 27	 25	 281	 1191	 271	
4.0	 0	 9	 5	 151	 3219	 1656	
5.0	 2	 5	 4	 25	 1547	 3689	


In [1]:
from sklearn.metrics import accuracy_score

accuracy_score(test_classes, s.classifier.predict(X_test))

NameError: name 'test_classes' is not defined

In [ ]:
import os
import pandas as pd
import numpy as np
import xmltodict
from xml.parsers.expat import ExpatError
from googletrans import Translator

In [ ]:
def translate(text):
    return Translator().translate(text, dest='pt')

norm = Preprocessing()

In [ ]:
# translate("Robert DeNiro plays the most unbelievably intelligent illiterate of all time. This movie is so wasteful of talent, it is truly disgusting. The script is unbelievable. The dialog is unbelievable. Jane Fonda's character is a caricature of herself, and not a funny one. The movie moves at a snail's pace, is photographed in an ill-advised manner, and is insufferably preachy. It also plugs in every cliche in the book. Swoozie Kurtz is excellent in a supporting role, but so what?<br /><br />Equally annoying is this new IMDB rule of requiring ten lines for every review. When a movie is this worthless, it doesn't require ten lines of text to let other readers know that it is a waste of time and tape. Avoid this movie.").text

In [ ]:
corpus_path = 'aclImdb/train/'

dataset_2 = {'polarity': [], 'bin_polarity':[], 'review':[], 'set':[]}

probability = 0.8

for folder in os.listdir(corpus_path):
    i = 0
    if "." not in folder and "unsup" not in folder:
        for file in os.listdir('{}{}/'.format(corpus_path, folder)):
            score = int(file.split('.')[0].split('_')[1])
            with open('{}{}/{}'.format(corpus_path, folder, file), 'r') as txt_file:
                comment = ""
                for line in txt_file.readlines():
                    comment += translate(line).text + " "
#                     print(comment)
#                     import time
#                     time.sleep(5)
                dataset_2['polarity'].append(score)
                dataset_2['bin_polarity'].append(0 if score <= 5 else 1)
                dataset_2['review'].append(comment)
                dataset_2['set'].append('train' if np.random.rand() < probability else 'test')
            i += 1
            print(i)
            if i == 2000: break
                

In [ ]:
dataset_2["review"]

In [ ]:


path = 'trainset/'

dataset = {'polarity':[], 'bin_polarity': [], 'review':[], 'set':[]}

probability = 0.8

for product in os.listdir(path):
    for score in os.listdir(path + product):
        for file in os.listdir(path + product + "/" + score + "/"):
            if file.endswith('.xml'):
                with open(path + product + "/" + score + "/" + file, 'r') as xml:
                    try:
                        data = xml.read()
                        data_dict = xmltodict.parse(data)
                        
                        if float(score) < 4:
#                             print("CAIU")
                            dataset['polarity'].append(float(data_dict['review']['stars']['@value']))
                            dataset['bin_polarity'].append(0 if float(data_dict['review']['stars']['@value']) < 3.0 else 1)
                            dataset['review'].append(data_dict['review']['cons'].replace('\n',''))
                            dataset['set'].append('train' if np.random.rand() < probability else 'test')
                        
                        # Comments
                        dataset['polarity'].append(float(data_dict['review']['stars']['@value']))
                        dataset['bin_polarity'].append(0 if float(data_dict['review']['stars']['@value']) < 3.0 else 1)
                        dataset['review'].append(data_dict['review']['opinion'].replace('\n',''))
                        dataset['set'].append('train' if np.random.rand() < probability else 'test')
                    except:
                        dataset['polarity'].pop(0)
                        dataset['bin_polarity'].pop(0)
                        
                            

In [ ]:
# dataset['review'] = dataset['review'][:-5]
# dataset['set'] = dataset['set'][:-5]

print(len(dataset['polarity']))
print(len(dataset['bin_polarity']))
print(len(dataset['review']))
print(len(dataset['set']))

In [ ]:
# create dataframe

dataframe = pd.DataFrame(data=dataset)
dataframe.groupby('polarity').count()

In [ ]:
import re
words = []

with open('LIWC2007_Portugues_win.dic.txt', 'r', encoding='latin') as liwc_file:
    in_header = True
    for line in liwc_file.readlines():
        if not re.match('^\d+', line):
            parts = line.split()
            word = parts.pop(0)
            if '126' in parts or '127' in parts:
                words.append(word)

In [ ]:
from franc_lib.lexical import Preprocessing
import spacy
spacy_nlp = spacy.load('pt_core_news_sm')
stopwords = spacy.lang.pt.stop_words.STOP_WORDS
# print(stopwords)

cont = 0
for word in words:
    if word in stopwords:
        stopwords.remove(word)

normalizer = Preprocessing()

def preprocessing(text):
    text = normalizer.lowercase(text)
    text = normalizer.remove_punctuation(text)
    tokens = normalizer.tokenize_words(text)
    tokens = [token for token in tokens if token not in stopwords]
    return ' '.join(tokens)

In [ ]:
dataframe['normalized_review'] = dataframe['review'].apply(preprocessing)
dataframe.head()

In [ ]:
# dataframe.head(100)

## Feature Extraction

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [ ]:
train_reviews = dataframe[dataframe['set'] == 'train']['normalized_review'].values.tolist()
train_classes = dataframe[dataframe['set'] == 'train']['polarity'].values.tolist()
test_reviews = dataframe[dataframe['set'] == 'test']['normalized_review'].values.tolist()
test_classes = dataframe[dataframe['set'] == 'test']['polarity'].values.tolist()

transformer = TfidfVectorizer()
transformer.fit(train_reviews)
X = transformer.transform(train_reviews)
X_test = transformer.transform(test_reviews)

# cv = CountVectorizer(binary=True)
# cv.fit(train_reviews)
# X = cv.transform(train_reviews)
# X_test = cv.transform(test_reviews)

## Import sklearn

In [ ]:
from sklearn.svm import SVR
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from sklearn.multiclass import OneVsRestClassifier

## Logistic Regression

In [ ]:
classifier = LogisticRegression(verbose=True)
classifier.fit(X, train_classes)

In [ ]:
accuracy_score(test_classes, classifier.predict(X_test))

In [ ]:
# svr = SVR()
# svr.fit(X, train_classes)

## MLP

In [ ]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(solver='sgd', hidden_layer_sizes=(50, 5), verbose=True)
# clf = MLPClassifier(solver='sgd'), n_jobs=4))

mlp.learning_rate_initial = 1

In [ ]:
mlp.fit(X, train_classes)

In [ ]:
accuracy_score(test_classes, mlp.predict(X_test))

## Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rf = RandomForestClassifier(n_estimators=100,n_jobs=3, criterion="entropy", verbose=1)
rf.fit(X, train_classes)

In [ ]:
accuracy_score(test_classes, rf.predict(X_test))

In [ ]:
sentence= ""
print(preprocessing(sentence))
X = transformer.transform([preprocessing(sentence)])
rf.predict(X)

## KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(15, weights='distance')
knn.fit(X, train_classes)

In [ ]:
accuracy_score(test_classes, knn.predict(X_test))